In [14]:
# !pip install sk-video

In [2]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import glob
import torch
import pandas as pd
import gc
from os.path import join as join_path

In [3]:
PATH_TO_FOLDER = '/home/jovyan/work/'
path = join_path(PATH_TO_FOLDER, 'run_attacks')

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device
DEVICE = device
print(device)

cuda:0


# Example of attack

In [18]:
# noise_file = f'{path}/uap/perturbations/lin_n.png'
# universal_noise = cv2.imread(noise_file)
# universal_noise = cv2.cvtColor(universal_noise, cv2.COLOR_BGR2RGB).astype('float32')
# universal_noise /= 255.
# universal_noise -= 0.5

# image_file = '/content/drive/MyDrive/colab/iqa_aimasters/data/val2017/000000001268.jpg'
# eps = 20 / 255
# image = cv2.imread(image_file)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('float32')
# image /= 255.
# h, w = image.shape[0], image.shape[1]

# universal_add = np.tile(universal_noise, (h//256 + 1, w//256 + 1, 1))[:h, :w, :]
# universal_add[universal_add > eps] = eps
# universal_add[universal_add < -eps] = -eps
# image_after_attack = universal_add + image
# image_after_attack[image_after_attack > 1] = 1
# image_after_attack[image_after_attack < 0] = 0
# image_after_attack = (image_after_attack * 255).astype('uint8')
# image_after_attack = cv2.cvtColor(image_after_attack, cv2.COLOR_RGB2BGR)
# cv2.imwrite(f'{path}/result.png', res)

# Run IQA models

In [19]:
# _type_folder = 'TID2013/distorted_images' # val2017 train2017 test2017 DIV2K_train_HR
_type_folder = 'KADID10K/kadid10k/images'
all_images = sorted(glob.glob(join_path(PATH_TO_FOLDER, 'data', _type_folder, '*')))
# image_type_folder = f'mscoco_{_type_folder}'
# image_type_folder = 'tid2013'
image_type_folder = 'kadid10k'
# image_type_folder = _type_folder

len(all_images)

10206

In [5]:
alltype_folder = 'TID2013/distorted_images' # val2017 train2017 test2017
_type_folder = 'train2017'

# _type_folder = 'KADID10K/kadid10k/images'
# image_type_folder = 'kadid10k'

all_images = [join_path(PATH_TO_FOLDER, 'data', _type_folder, '000000009898.jpg'), join_path(PATH_TO_FOLDER, 'data', _type_folder, '000000333018.jpg')]
image_type_folder = f'mscoco_{_type_folder}'
# image_type_folder = _type_folder
# image_type_folder = 'tid2013'

In [20]:
# df = pd.read_csv(join_path(PATH_TO_FOLDER, 'data', 'KADID10K', 'images_HackPredictModelV31_padding.csv'))
# df['distortion_type'] = df['image_name'].apply(lambda x: int(x.split('_')[1]) if len(x) != 7 else 0)
# ims = list(set(df[df['distortion_type'] == 6]['image_name'])) + list(set(df[df['distortion_type'] == 22]['image_name']))
# all_images = [join_path(PATH_TO_FOLDER, 'data', 'KADID10K', 'kadid10k', 'images', i) for i in ims]
# all_images = sorted(all_images)

# image_type_folder = 'kadid10k_only_6_22'

# len(all_images)

## Linearity

In [6]:
from argparse import ArgumentParser
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import resize, to_tensor, normalize
from PIL import Image
import h5py

In [7]:
def attack_image(image_file, universal_noise, eps=20./255.):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('float32')
    image /= 255.
    h, w = image.shape[0], image.shape[1]
    universal_add = np.tile(universal_noise, (h//256 + 1, w//256 + 1, 1))[:h, :w, :]
    universal_add[universal_add > eps] = eps
    universal_add[universal_add < -eps] = -eps

    image_after_attack = universal_add + image
    image_after_attack[image_after_attack > 1] = 1
    image_after_attack[image_after_attack < 0] = 0
    image_after_attack = (image_after_attack * 255).astype('uint8')
    image_after_attack = cv2.cvtColor(image_after_attack, cv2.COLOR_RGB2BGR)
    return image_after_attack

In [8]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'Linearity'))
from IQAmodel import IQAModel

In [ ]:
model = IQAModel().to(device)
model_path = '/home/jovyan/storage/NR-metric-models/p1q2.pth'
checkpoint = torch.load(model_path)
k = checkpoint['k']
b = checkpoint['b']
model.load_state_dict(checkpoint['model'])
model.eval();

In [10]:
noise_file = join_path(path, 'uap', 'perturbations', 'lin_n.png')
universal_noise = cv2.imread(noise_file)
universal_noise = cv2.cvtColor(universal_noise, cv2.COLOR_BGR2RGB).astype('float32')
universal_noise /= 255.
universal_noise -= 0.5
universal_noise.shape

(256, 256, 3)

In [11]:
eps = 20 / 255

In [12]:
fname = f'uap_{int(eps * 255)}_255_linearity_{image_type_folder}.csv'
fname

'uap_20_255_linearity_mscoco_train2017.csv'

In [17]:
# %%time
res = []
for image_file in tqdm(all_images[:1]):
    im = Image.open(image_file).convert('RGB')
    im = resize(im, (498, 664))
    im = to_tensor(im).to(device)
    im = normalize(im, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    with torch.no_grad():
      y = model(im.unsqueeze(0))[-1].cpu().detach().item()
      before_score = y * k[-1] + b[-1]

    image_after_attack = attack_image(image_file, universal_noise)
    im_pil = Image.fromarray(cv2.cvtColor(image_after_attack, cv2.COLOR_BGR2RGB))
    im = im_pil.convert('RGB')
    im = resize(im, (498, 664))
    im = to_tensor(im).to(device)
    im = normalize(im, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    with torch.no_grad():
        y = model(im.unsqueeze(0))[-1].cpu().detach().item()
        after_score = y * k[-1] + b[-1]
    res.append([image_file.split('/')[-1], before_score, after_score])

df = pd.DataFrame(res, columns =['image_name', 'before_score', 'after_score'])
df['metric'] = 'lin'
df['attack_type'] = f'uap_{int(eps * 255)}_255'
df.shape

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.25it/s]


(1, 5)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

## Paq-2-Piq

In [ ]:
noise_file = join_path(path, 'uap', 'perturbations', 'p2p_n.png')
universal_noise = cv2.imread(noise_file)
universal_noise = cv2.cvtColor(universal_noise, cv2.COLOR_BGR2RGB).astype('float32')
universal_noise /= 255.
universal_noise -= 0.5
universal_noise.shape

In [ ]:
os.environ['my_device'] = str(device)

os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'Paq-2-Piq'))
from paq2piq_standalone import *

In [ ]:
model_path = join_path(PATH_TO_FOLDER, 'metrics', 'Paq-2-Piq', 'RoIPoolModel.pth')
model = InferenceModel(RoIPoolModel().to(device), model_path)

In [ ]:
eps = 10.0 / 255.0

In [ ]:
# %%time
res = []
gains = []
for image_file in tqdm(all_images): # all_images
    im = cv2.imread(image_file)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    before_score = model.predict(im)['global_score'] # or score = model.predict_from_file(path_to_img)

    image = im.astype('float32')
    image /= 255.
    h, w = image.shape[0], image.shape[1]
    universal_add = np.tile(universal_noise, (h//256 + 1, w//256 + 1, 1))[:h, :w, :]
    universal_add[universal_add > eps] = eps
    universal_add[universal_add < -eps] = -eps

    image_after_attack = universal_add + image
    image_after_attack[image_after_attack > 1] = 1
    image_after_attack[image_after_attack < 0] = 0
    image_after_attack = (image_after_attack * 255).astype('uint8')
    after_score = model.predict(image_after_attack)['global_score']
    gains.append([before_score, after_score])
    
    res.append([image_file.split('/')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'uap_{int(eps * 255)}_255'
df['metric'] = 'p2p'

In [ ]:
torch.cuda.empty_cache()
gc.collect()

## SPAQ

In [ ]:
from PIL import Image
import torchvision
from torchvision import transforms

In [ ]:
noise_file = join_path(path, 'uap', 'perturbations', 'spaq_n.png')

universal_noise = cv2.imread(noise_file)
universal_noise = cv2.cvtColor(universal_noise, cv2.COLOR_BGR2RGB).astype('float32')
universal_noise /= 255.
universal_noise -= 0.5
universal_noise.shape

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'SPAQ'))
from Prepare_image import Image_load

class Baseline(torch.nn.Module):
	def __init__(self):
		super(Baseline, self).__init__()
		self.backbone = torchvision.models.resnet50(pretrained=False)
		fc_feature = self.backbone.fc.in_features
		self.backbone.fc = torch.nn.Linear(fc_feature, 1, bias=True)

	def forward(self, x):
		result = self.backbone(x)
		return result

model = Baseline()

model_path = '/home/jovyan/storage/NR-metric-models/BL_release.pt'
checkpoint = torch.load(model_path, device)
model.load_state_dict(checkpoint['state_dict'])
model.to(device);
model.eval();

prepare_image = Image_load(size=512, stride=224)

to_tensor = transforms.ToTensor()

In [ ]:
eps = 20 / 255
eps

In [ ]:
# %%time
res = []
for image_file in tqdm(all_images):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype('float32')
    try:
        img = prepare_image(Image.open(image_file).convert("RGB")).to(device)
    except:
        res.append([image_file.split('/')[-1], before_score, after_score])
        continue

    with torch.no_grad():
        before_score = model(img).mean().detach().cpu().item()

    image /= 255.
    h, w = image.shape[0], image.shape[1]
    universal_add = np.tile(universal_noise, (h//224 + 1, w//224 + 1, 1))[:h, :w, :]
    universal_add[universal_add > eps] = eps
    universal_add[universal_add < -eps] = -eps
    image_after_attack = universal_add + image
    image_after_attack[image_after_attack > 1] = 1
    image_after_attack[image_after_attack < 0] = 0
    image_after_attack = (image_after_attack * 255).astype('uint8')
    img = prepare_image(Image.fromarray(image_after_attack).convert("RGB")).to(device)
    # img = to_tensor(Image.fromarray(image_after_attack)).unsqueeze(0).to(device)
    with torch.no_grad():
        after_score = model(img).mean().detach().cpu().item()

    res.append([image_file.split('/')[-1], before_score, after_score])


df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'uap_{int(eps * 255)}_255'
df['metric'] = 'spaq'

In [ ]:
torch.cuda.empty_cache()
gc.collect()

## KonCept512

In [ ]:
import torch.nn as nn
import torchvision
from PIL import Image
import argparse
import os
from torchvision.transforms import ToTensor, Resize, Normalize
from torchvision import transforms

In [ ]:
noise_file = join_path(path, 'uap', 'perturbations', 'koncept_n.png')

universal_noise = cv2.imread(noise_file)
universal_noise = cv2.cvtColor(universal_noise, cv2.COLOR_BGR2RGB).astype('float32')
universal_noise /= 255.
universal_noise -= 0.5
universal_noise.shape

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'KonCept512'))

from inceptionresnetv2 import inceptionresnetv2

In [ ]:
class model_qa(nn.Module):
    def __init__(self,num_classes,**kwargs):
        super(model_qa,self).__init__()
        base_model = inceptionresnetv2(num_classes=1000, pretrained='imagenet')
        self.base = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(1536, 2048),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.25),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.25),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),         
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes),
        )

    def forward(self,x):
        x = self.base(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x    

In [ ]:
KonCept512 = model_qa(num_classes=1)
KonCept512.load_state_dict(torch.load('/home/jovyan/storage/NR-metric-models/KonCept512.pth'))
KonCept512.eval().to(device);

In [ ]:
koncept_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((384, 512)),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

In [ ]:
def img_to_batch_koncept(img):
    img_batch = torch.zeros(1, 3, 384, 512).to(device)  
    img_batch[0]  = koncept_transform(img) 
    return img_batch

In [ ]:
eps = 26 / 255

In [ ]:
%%time
res = []
gains = []
for image_file in tqdm(all_images):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    with torch.no_grad():
        before_score = KonCept512(img_to_batch_koncept(image).to(device)).detach().detach().cpu().numpy()[0][0]
    image = image.astype('float32')
    image /= 255.
    h, w = image.shape[0], image.shape[1]
    universal_add = np.tile(universal_noise, (h//300 + 1, w//300 + 1, 1))[:h, :w, :]
    universal_add[universal_add > eps] = eps
    universal_add[universal_add < -eps] = -eps
    image_after_attack = universal_add + image
    image_after_attack[image_after_attack > 1] = 1
    image_after_attack[image_after_attack < 0] = 0
    image_after_attack = (image_after_attack * 255).astype('uint8')
    with torch.no_grad():
        after_score = KonCept512(img_to_batch_koncept(image_after_attack).to(device)).detach().detach().cpu().numpy()[0][0]
    
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'uap_{int(eps * 255)}_255'
df['metric'] = 'koncept'

## MDTVSFA

In [ ]:
def attack_image(image_file, universal_noise, eps=20./255.):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype('float32')
    image /= 255.
    h, w = image.shape[0], image.shape[1]
    universal_add = np.tile(universal_noise, (h//256 + 1, w//256 + 1, 1))[:h, :w, :]
    universal_add[universal_add > eps] = eps
    universal_add[universal_add < -eps] = -eps

    image_after_attack = universal_add + image
    image_after_attack[image_after_attack > 1] = 1
    image_after_attack[image_after_attack < 0] = 0
    image_after_attack = (image_after_attack * 255).astype('uint8')
    image_after_attack = cv2.cvtColor(image_after_attack, cv2.COLOR_RGB2BGR)
    return image_after_attack

In [ ]:
noise_file = join_path(path, 'uap', 'perturbations', 'mdtvsfa_n.png')
universal_noise = cv2.imread(noise_file)
universal_noise = cv2.cvtColor(universal_noise, cv2.COLOR_BGR2RGB).astype('float32')
universal_noise /= 255.
universal_noise -= 0.5
universal_noise.shape

In [ ]:
import torch
import random
from torchvision import transforms
import skvideo.io
from PIL import Image
import numpy as np
import torch.nn as nn

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.utils.backcompat.broadcast_warning.enabled = True

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'metrics', 'MDTVSFA'))

from VQAmodel import VQAModel
from CNNfeatures import CNNModel

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
to_tensor = transforms.ToTensor()

In [ ]:
model = VQAModel().to(device)
model.load_state_dict(torch.load('/home/jovyan/storage/NR-metric-models/MDTVSFA.pt'))

In [ ]:
# get features
extractor = CNNModel(model='ResNet-50').to(device)

In [ ]:
model.train();
extractor.eval();

In [ ]:
eps = 9 / 255

In [ ]:
res = []
gains = []
i = 0
for image_file in tqdm(all_images):
    image = np.array(Image.open(image_file).convert('RGB'))
    with torch.no_grad():
        transformed_video = transform(image).unsqueeze(0)
        features_mean, features_std = extractor(transformed_video.to(device))
        features = torch.cat((features_mean, features_std), 1).squeeze()
        features = torch.unsqueeze(features, 0)
        input_length = features.shape[1] * torch.ones(1, 1, dtype=torch.long)

        before_score = model((features, input_length))
        before_score = before_score[0].detach().cpu().item()
        
    
    image_after_attack = attack_image(image_file, universal_noise, eps=eps)
    image_after_attack = Image.fromarray(cv2.cvtColor(image_after_attack, cv2.COLOR_BGR2RGB))
    with torch.no_grad():
        transformed_video = transform(image_after_attack).unsqueeze(0)
        features_mean, features_std = extractor(transformed_video.to(device))
        features = torch.cat((features_mean, features_std), 1).squeeze()
        features = torch.unsqueeze(features, 0)
        input_length = features.shape[1] * torch.ones(1, 1, dtype=torch.long)
        
        after_score = model((features, input_length))
        after_score = after_score[0].detach().cpu().item()
        
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])

gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'uap_{int(eps * 255)}_255'
df['metric'] = 'mdtvsfa'